In [28]:
import numpy as np
import numpy.random as rn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from math import log, factorial

In [30]:
vax_data = pd.read_csv('/Users/giovannimaya/Downloads/data/PCC_study_train.csv')
vax_data

,Unnamed: 0,vax_status,WBscore,PCCsymp
0,1,Vaccinated,Improved,Less
1,2,Vaccinated,Unchanged,Less
2,3,Vaccinated,Improved,Less
3,4,Vaccinated,Worsened,Less
4,5,Vaccinated,Worsened,More
...,...,...,...,...
70,71,Unvaccinated,Unchanged,Less
71,72,Unvaccinated,Worsened,Same
72,73,Unvaccinated,Improved,Same
73,74,Unvaccinated,Unchanged,More


In [35]:
# q1 

# a. WBScore and PCCsymp are categorical variables. 
WBscore_mapping = {'Improved': 0, 'Unchanged': 1, 'Worsened': 2}
PCCsymp_mapping = {'Less': 0, 'Same': 1, 'More': 2}
vax_status_mapping = {'Vaccinated': 0, 'Unvaccinated': 1}
vax_data = pd.read_csv('/Users/giovannimaya/Downloads/data/PCC_study_train.csv')
vax_data = vax_data.drop(columns=['Unnamed: 0'], axis=1)
vax_data['vax_status'] =  vax_data['vax_status'].map(vax_status_mapping)
vax_data['WBscore'] = vax_data['WBscore'].map(WBscore_mapping)
vax_data['PCCsymp'] = vax_data['PCCsymp'].map(PCCsymp_mapping)
vax_data


,vax_status,WBscore,PCCsymp
0,0,0,0
1,0,1,0
2,0,0,0
3,0,2,0
4,0,2,2
...,...,...,...
70,1,1,0
71,1,2,1
72,1,0,1
73,1,1,2


In [36]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.formula.api import logit


train_data, test_data = train_test_split(vax_data, test_size=0.2, random_state=42)
formula = 'vax_status ~ WBscore + PCCsymp'
model = logit(formula=formula, data = train_data).fit()
print(model.summary())

# b ~ The variables that are significant are the PCCsymp evaluation. 

Optimization terminated successfully.
         Current function value: 0.457020
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             vax_status   No. Observations:                   60
Model:                          Logit   Df Residuals:                       57
Method:                           MLE   Df Model:                            2
Date:                Mon, 08 Apr 2024   Pseudo R-squ.:                  0.3359
Time:                        12:29:14   Log-Likelihood:                -27.421
converged:                       True   LL-Null:                       -41.288
Covariance Type:            nonrobust   LLR p-value:                 9.497e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.3305      0.469     -2.839      0.005      -2.249      -0.412
WBscore        0.9504      0.

In [45]:
y = vax_data['vax_status']
X = vax_data[['WBscore', 'PCCsymp']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
all_cols = ' + '.join(vax_data.columns[1:])
formula = 'vax_status ~ ' + all_cols 
print("Formula: ", formula, '\n')

Formula:  vax_status ~ WBscore + PCCsymp 



In [48]:
train_data = pd.concat([X_train, y_train], axis=1)
model = logit(formula, data=train_data).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.457020
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             vax_status   No. Observations:                   60
Model:                          Logit   Df Residuals:                       57
Method:                           MLE   Df Model:                            2
Date:                Mon, 08 Apr 2024   Pseudo R-squ.:                  0.3359
Time:                        12:41:46   Log-Likelihood:                -27.421
converged:                       True   LL-Null:                       -41.288
Covariance Type:            nonrobust   LLR p-value:                 9.497e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.3305      0.469     -2.839      0.005      -2.249      -0.412
WBscore        0.9504      0.

In [ ]:
## start of c 